In [6]:
import thulac
import os
import re
import pandas as pd
from gensim.models import Word2Vec
import jieba
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import pandas as pd

In [2]:
# 停词表

def load_stop_word():
    stop_word = ['的', '。', '是', ' ', '\n', '日', '月', '.', '%','\u3000']
    stop_word_file = 'c:/data/stopwords'
    stop_word_dirs = os.listdir(stop_word_file)
    for textName in stop_word_dirs:
        filename = stop_word_file + '/' + textName
        with open(filename, 'r', encoding='utf-8') as file_object:
            line = file_object.readline()
            while line:
                stop_word.append(line[0:-1])
                line = file_object.readline()
    return stop_word

stop_word = load_stop_word()
print(len(stop_word))
print(stop_word[0:100])

3908
['的', '。', '是', ' ', '\n', '日', '月', '.', '%', '\u3000', '--', '?', '“', '”', '》', '－－', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', "a's", 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'came', 'can', 'cannot', 'cant', "can't", 'cause', 'causes', 'certain']


In [4]:
stop_word = load_stop_word()
print(len(stop_word))
print(stop_word[0:100])


# 2.5 分词前修改thulac源码中的bug

from contextlib import contextmanager
from io import StringIO
import time


@contextmanager
def add_clock_method_to_time():
    py_gt_3_8 = not hasattr(time, "clock")
    if py_gt_3_8:
        setattr(time, "clock", time.perf_counter)
    try:
        yield
    finally:
        if py_gt_3_8:
            delattr(time, "clock")
            
#分词
 # 默认模式，分词的同时进行词性标注
# thu = thulac.thulac(filt=True, seg_only=True)
word_type = {}
def cut_doc(doc):
     # 默认模式，分词的同时进行词性标注
    
    words = jieba.cut(doc)
    words_in_doc = ""
    for word in words:
        # print(word)
        if word in stop_word: continue
        words_in_doc +=  word + " "
        # word_type[word] = wordType
        
    return words_in_doc

words_in_doc = cut_doc("用来控制训练的文章数量")
words_in_doc

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hys\AppData\Local\Temp\jieba.cache


3908
['的', '。', '是', ' ', '\n', '日', '月', '.', '%', '\u3000', '--', '?', '“', '”', '》', '－－', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', "a's", 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'came', 'can', 'cannot', 'cant', "can't", 'cause', 'causes', 'certain']


Loading model cost 0.529 seconds.
Prefix dict has been built successfully.


'控制 训练 文章 数量 '

In [7]:
doc_map = {}
label_map = {}
map = {}
DOC_COUNT_MAX = 200
wrong_num = []

def read_file_cut(doc_file_name, doc_id, label_index):
    with open(doc_file_name, 'r', encoding='utf-8') as file_object:
        doc_content = ""
        line = file_object.readline()
        while line:
            doc_content += line
            line = file_object.readline()
        wrong = 1
        for _ in range(0,4):
            try:
                doc_map[doc_id] = cut_doc(doc_content)
                wrong = 0
                break
            except Exception as e:
                continue
        if wrong == 1:
            wrong_num.append(doc_id)
        label_map[doc_id] = int(label_index)
    return 0

my_df = pd.DataFrame([],index=[],columns=['text','label'])

label_num = {}

with ThreadPoolExecutor(max_workers= int(10000)) as t:
    obj_list = []
    begin = time.time()
    
    news_file = 'c:/data/THUCNews'
    news_file_dirs = os.listdir(news_file)
    # print(news_file_dirs)
    
    label_index = -1
    for label in news_file_dirs:
        label_index += 1
        label_num[label] = label_index
        
        file = news_file + '/' + label
        doc_ids = os.listdir(file)
        doc_num = 0
        for doc_name in doc_ids:
            doc_num += 1
            if doc_num >= DOC_COUNT_MAX:
                break
                
            doc_file_name = file + '/' + doc_name
            doc_id = doc_name[0:-4]
            doc_content = ""
            
            
            obj = t.submit(read_file_cut, doc_file_name, doc_id, label_index)
            obj_list.append(obj)    
            
      
    for future in as_completed(obj_list):
        data = future.result()
    times = time.time() - begin
    print("多线程占据的全部时间"+str(times))
    print(len(obj_list))
    print(len(doc_map))
    print(len(label_map))
    
map['text'] = doc_map
map['label'] = label_map


my_df = pd.DataFrame(map)
train_df = my_df
print("错误的次数", len(wrong_num) )
print(train_df.shape)
train_df.to_csv("c:/data/news/train_data301.csv")


多线程占据的全部时间73.79589986801147
2786
2786
2786
错误的次数 0
(2786, 2)


In [9]:
import numpy as np
useless = set()

doc_data = train_df['text']
common_text = []
for i in range(0, doc_data.size):
    doc = str(doc_data[i]).split(" ")
    common_text.append(doc)

# 词向量的维度，样本够的话300-500。
# min_count 最小次数计数，出现次数低于这个数的就不要了
# workers  线程数
# window 窗口大小
# sample 负例采样设置aa


vector_size = 500
model = Word2Vec(sentences=common_text, vector_size=vector_size, window=5, min_count=4,
                  workers=12, sample=1e-5)
# model.save("word2vec.model")
vector = model.wv['中国']

# 词向量 合成 文本向量
doc_v_list = np.array([])

for split_doc in common_text:
    used_word = float(0.0)
    vector = vector * 0
    for word in split_doc:

        used_word += 1.0
        try:
            vector += model.wv[word]
        except Exception as e:
            # print(e.__traceback__.tb_frame.f_globals["__file__"])  # 发生异常所在的文件
            # print(e.__traceback__.tb_lineno)  # 发生异常所在的行数
            used_word -= 1.0

    vector /= used_word /200
    doc_v_list = np.append(doc_v_list, vector)

train_test = doc_v_list.reshape(len(common_text), vector_size)


In [80]:
# 基于每个文档的词语进行聚类





j = 0
ClusterNum = 6

for doc in train_df['text']:
   

    used_word = 0
    word_matrix =  np.array([])
    # print(doc)
    index_of_word_map = {}
    doc = str(doc)
#     print(doc.split())
    for word in doc.split():
        if len(word) < 2: 
            continue
        try:
            word_matrix = np.append(word_matrix, model.wv[word])
            index_of_word_map[used_word] = word
            used_word += 1
            
        except Exception as e:
            pass
        
    cluster_test = word_matrix.reshape(used_word, vector_size)
    
    
    ##################### SNOWNLP ##############
    from snownlp import SnowNLP
    s = SnowNLP(doc)
    print(s.keywords(ClusterNum))
    
    
    
    
    
    

    
    
    ################### KMEANS #####################
    
    
    
    from sklearn.cluster import KMeans
    k_means =  KMeans(n_clusters=ClusterNum, random_state=9)
    y_pred = k_means.fit_predict(cluster_test)



    best_key_set = set()
    for center in k_means.cluster_centers_:
        # print(type(center))

        best_key = 0
        min_dist = np.sqrt(np.sum((word_vector - cluster_test[0]) ** 2))
        
        for key in range(0, len(cluster_test)):
            if key in best_key_set: continue
            word_vector = cluster_test[key]
            #print(word_vector)
            dist = np.sqrt(np.sum((word_vector - center) ** 2))
            if dist < min_dist:
                #print(dist)
                best_key = key

        best_key_set.add(best_key)
        print(best_key, index_of_word_map[best_key])
    
    j += 1
    print("*************")
    if j > 10:
        break
    
# word_num, wrong_word_num, used_word

['训练', '沈阳', '国奥队', '大雨', '中', '球员']
175 青睐
176 不解
174 雨水
173 球员
166 大雨
172 国奥
*************
['姚明', '罗萨斯', '场', '球队', '火箭', '说']
177 之上
178 肥仔
176 轨道
168 时机
166 情况
175 正确
*************
['易', '建', '联', '尼克斯', '中', '球馆']
215 麦迪逊
227 机会
229 方生
225 易建联
224 体验
228 春水
*************
['姚', '说', '昨天', '上', '球队', '德尔曼']
180 火山
176 取胜
177 休斯敦
172 球队
179 时报
178 华夏
*************
['科', '湖', '珀森', '球员', '上', '人']
119 不错
100 乐于
117 球员
63 一员
78 坐在
87 工作
*************
['李', '分', '机会', '赛', '火箭', '说']
106 马刺
127 华夏
120 以往
129 火山
96 表现
122 担纲
*************
['普里', '比拉', '兹', '队', '开拓者', '复']
190 奥登
185 开拓者
194 一支
162 感觉
196 一名
192 比拉
*************
['詹姆斯', '训练', '中', '右腿', '疼痛', '比赛']
93 能量
94 小林
92 释放
80 一堂
90 100%
91 比赛
*************
['热火', '主场', '篮网', '出', '墨', '菲']
60 奥特
59 莫罗
57 篮网
58 哈里斯
48 详细
43 好头
*************
['辽足', '俱乐部', '球员', '合同', '超', '工资']
323 态度
322 坦诚
321 俱乐部
320 辽足
0 辽足
319 球员
*************
['菲', '韩', '男篮', '新', '写真', '中']
108 天下
107 红遍
57 真的
106 写真
104 性感
60 发自内心
*************


In [87]:


####################### TF-IDF #############

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score


tfidf = TfidfVectorizer(ngram_range=(1,4), max_features=300)
train_test = tfidf.fit_transform(train_df['text'])


C:\Users\hys\AppData\Local\Temp\ipykernel_19916\3842295331.py:15: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  keyWords = pd.Index(names)[sort]


array([['球队', '31', '球员'],
       ['防守', '火箭', '姚明'],
       ['表现', '篮板', '比赛'],
       ['时间', '姚明', '球队'],
       ['调整', '这一', '球员'],
       ['比赛', '火箭', '机会'],
       ['情况', '主场', '球队'],
       ['100', '客场', '比赛'],
       ['详细', '比赛', '主场'],
       ['球队', '赛季', '球员']], dtype=object)

In [89]:
sort = np.argsort(train_test.toarray(), axis = 1)[:, -10:]
names = tfidf.get_feature_names()
keyWords = pd.Index(names)[sort]
keyWords[0:21]

C:\Users\hys\AppData\Local\Temp\ipykernel_19916\4121189783.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  keyWords = pd.Index(names)[sort]


array([['这一', '位置', '27', '29', '事情', '一位', '25', '球队', '31', '球员'],
       ['这是', '未来', '采访', '比赛', '原因', '表现', '球队', '防守', '火箭', '姚明'],
       ['这是', '2009', '依然', '优势', '球队', '赛季', '时间', '表现', '篮板', '比赛'],
       ['采访', '游戏', '回答', '球员', '火箭', '记者', '11', '时间', '姚明', '球队'],
       ['很大', '一位', '找到', '不错', '一名', '防守', '特别', '调整', '这一', '球员'],
       ['只能', '对手', '回答', '进攻', '时间', '两个', '结束', '比赛', '火箭', '机会'],
       ['很大', '10', '压力', '26', '赛季', '12', '11', '情况', '主场', '球队'],
       ['感觉', '参加', '媒体', '状态', '面对', '球队', '详细', '100', '客场', '比赛'],
       ['新浪', '北京', '11', '这是', '北京 时间', '对手', '客场', '详细', '比赛', '主场'],
       ['几个', '情况', '20', '2008', '24', '2009', '管理', '球队', '赛季', '球员'],
       ['希望', '关注', '方式', '16', '参加', '真的', '依然', '优势', '近期', '一年'],
       ['冠军', '本场', '10', '12', '表现', '16', '17', '篮板', '球队', '比赛'],
       ['接受', '北京 时间', '第一', '不错', '采访', '这是', '表现', '一点', '球员', '球队'],
       ['时间', '最终', '只能', '机会', '表现', '球队', '选择', '比赛', '进攻', '防守'],
       ['记者', '美国', '

In [52]:
k_means.cluster_centers_

array([[ 0.08378126, -0.1839826 ,  0.16146725, ...,  0.02087184,
        -0.13922298, -0.30269699],
       [ 0.05630706, -0.12312471,  0.10766536, ...,  0.01407618,
        -0.09354932, -0.2032135 ],
       [ 0.02327915, -0.05024099,  0.04506252, ...,  0.00531093,
        -0.03750145, -0.08308181],
       [ 0.13345828, -0.29105409,  0.25545619, ...,  0.03428826,
        -0.22079782, -0.47747483]])

球员
190
不解
193
青睐
192
中
165
